DATASET DEVELOPMENT AND FEATURE SELECTION

In [ ]:
# Importing Libraries
import numpy as np
import pandas as pd

# read dataset
df = pd.read_csv(r"C:\Users\Akash\Downloads\run\run_500\P_N.csv")

font_size = 18

positive = df[df['Label'] == 1]
negative = df[df['Label'] == 0]

#plotting the "Sequence" column array length for 0 and 1 labels in the df dataframe with 1200 pixels in width and 600 pixels in height
import matplotlib.pyplot as plt
plt.hist([positive['Sequence'].str.len(), negative['Sequence'].str.len()], bins=40, label=['Positive', 'Negative'])

#setting the legend and font size of the plot
plt.legend(fontsize=font_size)

# setting the x and y axis labels
plt.xlabel('Sequence Length', fontsize=font_size)
plt.ylabel('Count', fontsize=font_size)

#setting the font of the x and y tick labels
plt.xticks(fontsize=font_size)
plt.yticks(fontsize=font_size)

figsize = (40, 30)


# saving the plot in 1200 dpi
plt.savefig(r"C:\Users\Akash\Downloads\run\run_500\0_Sequence_Distribution.png", dpi=1200, bbox_inches='tight')


# Developing X and Y
X = df.drop(['ID', 'Label', 'Sequence'], axis=1)
Y = df[['Label']]
Y = np.ravel(Y)

cols_MI = X.columns


#plotting X max values for each column
X.max().plot(kind='bar')


# isolating the X columns with maximum value greater than 1.0
df1 = X.loc[:, (X.max() > 1.0)]

#develop a for loop to divide each value in column by 100
for i in df1.columns:
    X[i] = df1[i].div(100)

df1.describe()
X.describe()

#plotting X max values for each column
X.max().plot(kind='bar')



In [ ]:
from sklearn.feature_selection import mutual_info_classif
import random
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.feature_selection import mutual_info_classif as MIC
import os

In [ ]:
def myfunction(X,Y, iteration):
    random_numbers = random.sample(range(0, 1000000), 500)
    mutual_info = []
    for i in random_numbers:
        mutual_info.append(mutual_info_classif(X,Y,random_state=i))
    
    #converting the list into a dataframe
    df_list = pd.DataFrame(mutual_info)
    #transposing the list to insert column names as index
    df_list = df_list.T
    df_list = df_list.set_index(cols_MI)
    df_list = df_list.T

    df_list_2 = df_list
    df_list_2 = df_list_2.T

    #Sorting by column "mean"
    df_list_2['mean'] = df_list_2.mean(axis=1)

    df_list_2 = df_list_2.sort_values(by=['mean'], ascending=False)

    #Filtering MI values greater than 0.030
    df_list_2 = df_list_2[df_list_2['mean'] > 0.03]
    #print(df_list_2)

    #Creating a subset of X
    MI_score_filtered_X = X[df_list_2.index]

    # creating new dataset for analysis
    Dataset_etc = df[['ID', 'Label', 'Sequence']]
    Dataset_MI_1 = pd.concat([Dataset_etc, MI_score_filtered_X], axis=1)
    # create a folder named iteration

    os.mkdir(r"C:\Users\Akash\Downloads\run\run_500\run_"+str(iteration))
    # save the dataset in this newly created folder

    Dataset_MI_1.to_csv(r"C:\Users\Akash\Downloads\run\run_500\run_"+str(iteration)+"\1_Dataset_MI_1.csv",index=False)


    mutual_info_features = []
    index_list = ['Run' + str(i + 1) for i in range(df_list.shape[0])]
    for i,row in df_list.iterrows():
        row_data = row.to_dict()
        row_data = dict(sorted(row_data.items(),key = lambda x:x[1],reverse=True))
        temp_list = [i for i in row_data.keys()]
        mutual_info_features.append(temp_list[:10])
    


    temp = {}
    for index,row in zip(index_list,mutual_info_features):
        temp[index] = row
    df_mi = pd.DataFrame(temp)
    df_mi = df_mi.T
    df_mi
    df_mi.to_csv(r"C:\Users\Akash\Downloads\run\run_500\run_"+str(iteration)+"\2_MI_Features.csv",index=False)
    

    #Ananlysis: 2 Ranking
    df_list_1 = df_list
    df_list_1 = df_list_1.T
    ranked_list = df_list_1.rank()

    ranked_list['mean'] = ranked_list.mean(axis=1)

    ranked_list = ranked_list.sort_values(by=['mean'], ascending=False)

    dataset_VIF = pd.read_csv(r"C:\Users\Akash\Downloads\run\run_500\run_"+str(iteration) + "\2_Dataset_MI_1.csv",index_col=0)

    # Developing X and filling the empty values
    X_VIF = dataset_VIF.drop(['ID', 'Label', 'Sequence'], axis=1)

    #transposing dataset_VIF
    X_VIF = X_VIF.T

    # Sorting the index of X_VIF in ascending order
    X_VIF = X_VIF.sort_index(axis=0, ascending=True)

    X_VIF = X_VIF.T

    def computeVIF(Xdata):
        # VIF dataframe
        vif_data = pd.DataFrame()
        vif_data["feature"] = Xdata.columns

        #print(vif_data)

        # calculating VIF for each feature
        vif_data["VIF"] = [variance_inflation_factor(Xdata.values, i) for i in range(len(Xdata.columns))]
        return vif_data
    
    #Computing VIF < 05
    my_lists_1 = [list() for i in range(300)]

    for i in range(len(my_lists_1)):
        temp_1 = computeVIF(X_VIF)
        temp_1 = temp_1.sort_values(by=["VIF"],ascending=False).reset_index(drop=True)
        if(temp_1['VIF'][0]>5):
            temp_1 = temp_1.iloc[1:]
            my_lists_1[i] = temp_1.loc[temp_1['VIF']<=5]['feature'].to_list()
            X_VIF=X_VIF[temp_1['feature']]
        else:
            break
    #X_VIF is getting updated by the features whose values are less than 10 because our compute vif
    #function is returning only those features whose values are less than 10 for the VIF score

    temp_1 = temp_1.sort_values(by="VIF", ascending=False)
    temp_1.head()
    temp_1.to_csv(r"C:\Users\Akash\Downloads\run\run_500\run_"+str(iteration)+"\5_VIF_values.csv",index=False)

    Xdata_after_MI_1_VIF_iter = X[temp_1['feature']]
    Xdata_after_MI_1_VIF_iter.shape

    # creating new dataset for analysis "dataset" to make, "Dataset" for new dataset.
    dataset_MI_1_VIF = pd.concat([Dataset_etc, Xdata_after_MI_1_VIF_iter], axis=1)
    dataset_MI_1_VIF.to_csv(r"C:\Users\Akash\Downloads\run\run_500\run_"+ str(iteration) +"\7_dataset_MI_1_VIF.csv",index=False)

    Dataset_MI_1_VIF = pd.read_csv(r"C:\Users\Akash\Downloads\run\run_500\run_"+ str(iteration) + "\7_dataset_MI_1_VIF.csv")

    X_MI_2 = Dataset_MI_1_VIF.drop(['ID', 'Label', 'Sequence'], axis=1)
    Y_MI_2 = Dataset_MI_1_VIF[['Label']]

    col_MI_2 = X_MI_2.columns
    #Calculating MI values
    MI_score_2 = MIC(X_MI_2,Y_MI_2, random_state =99)
    col_MI_2 = list(col_MI_2)
    #Creating dataframe with two lists mi_score with heading "MI Scores" and col list with heading "Attributes"
    MI_score_2_df = pd.DataFrame({'MI Scores': MI_score_2, 'Attributes': col_MI_2})
    MI_score_2_df.to_csv(r"C:\Users\Akash\Downloads\run\run_500\run_"+ str(iteration)+ "\8_MI_Score_2.csv",index=False)


In [ ]:
import threading

# run myfunction 10 times

for i in range(10):
    t = threading.Thread(target=myfunction, args=(X,Y,i+1))
    t.start()
    t.join()
    print("Thread", i+1, "is completed")



In [ ]:
#creating a list 
selected_features = [list() for i in range(10)]
#create a for loop to read the csv files

for i in range(10):
    #read the csv files
    print(i+1)
    dataframe = pd.read_csv("./run_"+ str(i+1) + "/8_MI_Score_2.csv")
    print(dataframe.shape)
    print(i)
    # adding these values to the list selected_feature as a new column
    selected_features[i] = dataframe['Attributes'].to_list()

for i in range(10):
    print(len(selected_features[i]))

In [ ]:
common_features = set.intersection(*map(set, selected_features))
len(common_features)
print(common_features)


In [ ]:
all_values = []
for i in range(10):
    all_values.extend(selected_features[i])
unique_values = set(all_values)
len(unique_values)
print(unique_values)

In [ ]:
print(type(unique_values))
print(type(common_features))
print(unique_values)

In [ ]:
dataset_etc = df[['ID', 'Label', 'Sequence']]

In [ ]:
# converting unique_values data type from set to list
unique_values = list(unique_values)

In [ ]:
print((unique_values))

In [ ]:
df = pd.read_csv("./P_N.csv")
print(df.columns)

In [ ]:
# creating a subset from dataframe df by using column name present in set of unique_values and matching them with the column names in df
df_subset_unique = df[unique_values]
df_subset_unique = pd.concat([dataset_etc, df_subset_unique], axis=1)
df_subset_unique.to_csv("./9_dataset_subset_unique.csv",index=False)
print(df_subset_unique.shape)

df_subset_common = df[common_features] 
df_subset_common = pd.concat([dataset_etc, df_subset_common], axis=1)
df_subset_common.to_csv("./10_dataset_subset_common.csv",index=False)
print(df_subset_common.shape)